In [1]:
'''
Notebook for FIGS3 - Unwanted effects of essenstial gene co-expression on cells with toggle switch circuits(compared to having only toggle switches in the cell)
'''
# By Kirill Sechkar

# PACKAGE IMPORTS 
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
from math import pi
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 

# OWN CODE IMPORTS
import synthetic_circuits as circuits
from cell_model import *

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


cpu


Loading BokehJS ...

In [2]:
# TOGGLE SWITCHES ONLY
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.punisher_b_initialise,
    circuits.punisher_b_ode,
    circuits.punisher_b_F_calc,
    par, init_conds)  # load the circuit

In [3]:
# TOGGLE SWITCHES ONLY
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par_only = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.twotoggles_punisher_initialise,
    circuits.twotoggles_punisher_ode,
    circuits.twotoggles_punisher_F_calc,
    par, init_conds)  # load the circuit

In [4]:
# TOGGLE SWITCHES ONLY
# PARAMETERISE THE CIRCUIT

# TOGGLE SWITCHES
for togswitchnum in (1, 2):  # cycle through toggle switches
    for toggenenum in (1, 2):  # cycle through the genes of the current switch
        par['c_tog' + str(togswitchnum) + str(toggenenum)] = 1  # copy no. (nM)
        par['a_tog' + str(togswitchnum) + str(toggenenum)] = 1e5/2  # promoter strength (unitless)

        # transcription regulation function
        reg_func_string = 'dna(tog' + str(togswitchnum) + str(toggenenum) + '):p_tog' + str(togswitchnum) + str(
            (toggenenum - 2) % 2 + 1)  # dna(rep1):p_rep3, dna(rep2):p_rep1 or dna(rep3):p_rep2
        par['K_' + reg_func_string] = 2500  # half-saturation constant
        par['eta_' + reg_func_string] = 2  # Hill coefficient
        par['baseline_tog' + str(togswitchnum) + str(
            toggenenum)] = 0.025  # baseline transcription activation due to leakiness
        par['p_tog' + str(togswitchnum) + str(
            toggenenum) + '_ac_frac'] = 1  # active fraction of protein (i.e. share of molecules not bound by the inducer)
    # break symmetry for each of the toggle switches
    init_conds['m_tog' + str(togswitchnum) + '1'] = 4000

# CULTURE MEDIUM
init_conds['s'] = 0.5
par['h_ext'] = 0 # no CAT as the essenbtial gene => no chloramphenicol added to the medium

# back up the system parameters
par_togglesonly = par.copy()
init_conds_togglesonly = init_conds.copy()

In [5]:
# TOGGLE SWITCHES ONLY
# SET UP THE DETERMINISTIC SIMULATION PARAMETERS

# diffrax simulator
savetimestep = 0.05  # save time step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver

# SIMULATION TIME FRAMES
# getting the initial state
tf_premut = (0,50+savetimestep/2) # time frame for simulation before the toggle genes  utate
tf_mut=(50,85+savetimestep/2)

In [6]:
# TOGGLE SWITCHES ONLY
# SIMULATE THE MUTATION OF ONE TOGGLE SWITCH GENE

par_onemut = par.copy()

 # initial simulation to get the steady state without gene expression loss
sol = ode_sim(par_onemut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_onemut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf_premut, jnp.arange(tf_premut[0], tf_premut[1], savetimestep),  # time frame and time axis for saving the system's state
              rtol, atol)  # relative and
ts = np.array(sol.ts)
xs = np.array(sol.ys)

par_onemut['func_tog11'] = 0.0 
x0_mut = xs[-1, :]  # simulation will resume from the last time point
sol_mut = ode_sim(par_onemut,  # dictionary with model parameters
                        ode_with_circuit,  # ODE function for the cell with synthetic circuit
                        x0_mut,  # initial condition VECTOR
                        len(circuit_genes), len(circuit_miscs), circuit_name2pos,  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                        cellmodel_auxil.synth_gene_params_for_jax(par_onemut, circuit_genes), # synthetic gene parameters for calculating k values
                        tf_mut, jnp.arange(tf_mut[0], tf_mut[1], savetimestep),  # time frame and time axis for saving the system's state
                        rtol, atol)  # relative and absolute tolerances
onemut_ts = np.concatenate((ts, np.array(sol_mut.ts)), axis=0)
onemut_xs = np.concatenate((xs, np.array(sol_mut.ys)), axis=0)

# get growth rates
_, onemut_ls, _, _, _, _, _, _ =cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(onemut_ts, onemut_xs, par_onemut, circuit_genes, circuit_miscs, circuit_name2pos)

In [11]:
# TOGGLE SWITCHES ONLY
# MUTATION OF ONE TOGGLE SWITCH GENE

# FIGS3A - TOGGLE PROTEIN LEVELS
onemut_togprot_figure = bkplot.figure(
        frame_width=200,
        frame_height=125,
        title="Toggle switches only",
        x_axis_label="Time since mutation, h",
        y_axis_label="Protein conc., nM",
        x_range=(-5,tf_mut[1]-tf_mut[0]),
        y_range=(0,1.2e5),
        tools="box_zoom,pan,hover,reset,save"
    )

# add shading to show when synthetic gene expression loss occurs
onemut_togprot_figure.add_layout(
    bkmodels.PolyAnnotation(xs=[0, 0, tf_mut[1] - tf_mut[0], tf_mut[1] - tf_mut[0]],
                            ys=[0, 3e5, 3e5, 0],
                            line_width=0, line_alpha=0,
                            fill_color=bkRGB(100, 100, 100, 0.25)))
onemut_togprot_figure.add_layout(bkmodels.Label(x=0, y=1.2e5,
                                x_offset=2, y_offset=-12,
                                text='tog11 gene mutated',
                                text_font_size='8pt',
                                text_align='left'))

# plot the toggle switch protein concentrations for tog11 and tog21
onemut_togprot_figure.line(onemut_ts - tf_mut[0], onemut_xs[:, circuit_name2pos['p_tog11']], line_width=2, line_color=bkRGB(100, 149, 237),
           legend_label='tog11')
onemut_togprot_figure.line(onemut_ts - tf_mut[0], onemut_xs[:, circuit_name2pos['p_tog21']], line_width=2, line_color=bkRGB(207, 181, 59),
           legend_label='tog21')

# plot the toggle switch protein concentrations for tog12 and tog22
onemut_togprot_figure.line(onemut_ts - tf_mut[0], onemut_xs[:, circuit_name2pos['p_tog12']], line_width=2, line_color=bkRGB(127, 255, 212),
           legend_label='tog12')
onemut_togprot_figure.line(onemut_ts - tf_mut[0], onemut_xs[:, circuit_name2pos['p_tog22']], line_width=2, line_color=bkRGB(252, 194, 0),
           legend_label='tog22')

# legend formatting
onemut_togprot_figure.legend.location = "bottom_right"
onemut_togprot_figure.legend.label_text_font_size = "8pt"
onemut_togprot_figure.legend.padding = 2
onemut_togprot_figure.legend.spacing = 1
onemut_togprot_figure.legend.glyph_width = 10
onemut_togprot_figure.legend.margin = 7


# set fonts
onemut_togprot_figure.xaxis.axis_label_text_font_size = "8pt"
onemut_togprot_figure.xaxis.major_label_text_font_size = "8pt"
onemut_togprot_figure.yaxis.axis_label_text_font_size = "8pt"
onemut_togprot_figure.yaxis.major_label_text_font_size = "8pt"
onemut_togprot_figure.yaxis[0].formatter = bkmodels.PrintfTickFormatter(format="%.1e")

# FIGS3B - GROWTH RATE
# get the reference growth rate
pos_premut = np.where(onemut_ts <= tf_mut[0])[0][-1]
l_ref = onemut_ls[pos_premut]

onemut_gr_figure = bkplot.figure(
        frame_width=200,
        frame_height=125,
        title="Toggle switches only",
        x_axis_label="Time since mutation, h",
        y_axis_label="Relative cell growth rate",
        x_range=(-5, tf_mut[1] - tf_mut[0]),
        y_range=(0.9, 1.05),
        tools="box_zoom,pan,hover,reset,save"
    )

onemut_gr_figure.add_layout(
    bkmodels.PolyAnnotation(xs=[0, 0, tf_mut[1] - tf_mut[0], tf_mut[1] - tf_mut[0]],
                            ys=[0, 1.1, 1.1, 0],
                            line_width=0, line_alpha=0,
                            fill_color=bkRGB(100, 100, 100, 0.25)))
onemut_gr_figure.add_layout(bkmodels.Label(x=0, y=1.05,
                                x_offset=2, y_offset=-12,
                                text='tog11 gene mutated',
                                text_font_size='8pt',
                                text_align='left'))
# plot the growth rate
onemut_gr_figure.line(onemut_ts - tf_mut[0], onemut_ls/l_ref, line_width=2, line_color=bkRGB(0, 0, 0), legend_label='Growth rate')

# set fonts
onemut_gr_figure.xaxis.axis_label_text_font_size = "8pt"
onemut_gr_figure.xaxis.major_label_text_font_size = "8pt"
onemut_gr_figure.yaxis.axis_label_text_font_size = "8pt"
onemut_gr_figure.yaxis.major_label_text_font_size = "8pt"

# legend formatting
onemut_gr_figure.legend.location = "top_right"
onemut_gr_figure.legend.label_text_font_size = "8pt"
onemut_gr_figure.legend.padding = 3
onemut_gr_figure.legend.spacing = 1
onemut_gr_figure.legend.glyph_width = 10
onemut_gr_figure.legend.margin = 2
onemut_gr_figure.legend.visible = False

# show plots
onemut_togprot_figure.output_backend = "svg"
onemut_gr_figure.output_backend = "svg"
bkplot.show(bklayouts.column([onemut_togprot_figure, onemut_gr_figure]))

In [12]:
# TOGGLE SWITCHES WITH SYNTHETIC ADDICTION
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.twotoggles_add_initialise,
    circuits.twotoggles_add_ode,
    circuits.twotoggles_add_F_calc,
    par, init_conds)  # load the circuit

In [13]:
# TOGGLE SWITCHES WITH SYNTHETIC ADDICTION
# PARAMETERISE THE CIRCUIT

# TOGGLE SWITCHES - same as in the toggle switch-only case
for togswitchnum in (1, 2):  # cycle through toggle switches
    for toggenenum in (1, 2):  # cycle through the genes of the current switch
        par['c_tog' + str(togswitchnum) + str(toggenenum)] = par_togglesonly['c_tog' + str(togswitchnum) + str(toggenenum)]
        par['a_tog' + str(togswitchnum) + str(toggenenum)] = par_togglesonly['a_tog' + str(togswitchnum) + str(toggenenum)]

        # transcription regulation function
        reg_func_string = 'dna(tog' + str(togswitchnum) + str(toggenenum) + '):p_tog' + str(togswitchnum) + str(
            (toggenenum - 2) % 2 + 1)  # dna(rep1):p_rep3, dna(rep2):p_rep1 or dna(rep3):p_rep2
        par['K_' + reg_func_string] = par_togglesonly['K_' + reg_func_string]
        par['eta_' + reg_func_string] = par_togglesonly['eta_' + reg_func_string]
        par['baseline_tog' + str(togswitchnum) + str(
            toggenenum)] = par_togglesonly['baseline_tog' + str(togswitchnum) + str(toggenenum)]
        par['p_tog' + str(togswitchnum) + str( toggenenum) + '_ac_frac'] = par_togglesonly['p_tog' + str(togswitchnum) + str(toggenenum) + '_ac_frac']
    # break symmetry for each of the toggle switches
    init_conds['m_tog' + str(togswitchnum) + '1'] = init_conds_togglesonly['m_tog' + str(togswitchnum) + '1']

# OVERLAPPED CAT GENES
for togswitchnum in (1, 2):  # cycle through toggle switches
    for toggenenum in (1, 2):  # cycle through the genes of the current switch
        par['k+_cat' + str(togswitchnum) + str(toggenenum)] = 60/250  # ribosome binding rate (/h/nM)

# CULTURE MEDIUM
init_conds['s'] = 0.5
par['h_ext'] = 10.5 * (10.0 ** 3)  # no CAT as the essential gene => add chloramphenicol to the medium

# back up the system parameters
par_togglesonly = par.copy()

In [14]:
# TOGGLE SWITCHES WITH SYNTHETIC ADDICTION
# SET UP THE DETERMINISTIC SIMULATION PARAMETERS

# diffrax simulator
savetimestep = 0.05  # save time step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver

# SIMULATION TIME FRAMES
# getting the initial state
tf_premut = (0, 50 + savetimestep / 2)  # time frame for simulation before the toggle genes  utate
tf_mut = (50, 85 + savetimestep / 2)
# TOGGLE SWITCHES ONLY
# SIMULATE THE MUTATION OF ONE TOGGLE SWITCH GENE

par_onemut = par.copy()

# initial simulation to get the steady state without gene expression loss
sol = ode_sim(par_onemut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_onemut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf_premut, jnp.arange(tf_premut[0], tf_premut[1], savetimestep),
              # time frame and time axis for saving the system's state
              rtol, atol)  # relative and
ts = np.array(sol.ts)
xs = np.array(sol.ys)

par_onemut['func_tog11'] = 0.0
x0_mut = xs[-1, :]  # simulation will resume from the last time point
sol_mut = ode_sim(par_onemut,  # dictionary with model parameters
                  ode_with_circuit,  # ODE function for the cell with synthetic circuit
                  x0_mut,  # initial condition VECTOR
                  len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                  cellmodel_auxil.synth_gene_params_for_jax(par_onemut, circuit_genes),
                  # synthetic gene parameters for calculating k values
                  tf_mut, jnp.arange(tf_mut[0], tf_mut[1], savetimestep),
                  # time frame and time axis for saving the system's state
                  rtol, atol)  # relative and absolute tolerances
onemut_ts = np.concatenate((ts, np.array(sol_mut.ts)), axis=0)
onemut_xs = np.concatenate((xs, np.array(sol_mut.ys)), axis=0)

# get growth rates
_, onemut_ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(onemut_ts, onemut_xs, par_onemut,
                                                                              circuit_genes, circuit_miscs,
                                                                              circuit_name2pos)

In [16]:
# TOGGLE SWITCHES WITH SYNTHETIC ADDICTION
# MUTATION OF ONE TOGGLE SWITCH GENE

# FIGS3C - TOGGLE PROTEIN LEVELS
onemut_togprot_figure = bkplot.figure(
    frame_width=200,
    frame_height=125,
    title="Toggle switches with co-expression",
    x_axis_label="Time since mutation, h",
    y_axis_label="Protein conc., nM",
    x_range=(-5, tf_mut[1] - tf_mut[0]),
    y_range=(0, 1.2e5),
    tools="box_zoom,pan,hover,reset,save"
)

# add shading to show when synthetic gene expression loss occurs
onemut_togprot_figure.add_layout(
    bkmodels.PolyAnnotation(xs=[0, 0, tf_mut[1] - tf_mut[0], tf_mut[1] - tf_mut[0]],
                            ys=[0, 3e5, 3e5, 0],
                            line_width=0, line_alpha=0,
                            fill_color=bkRGB(100, 100, 100, 0.25)))
onemut_togprot_figure.add_layout(bkmodels.Label(x=0, y=1.2e5,
                                                x_offset=2, y_offset=-12,
                                                text='tog11 gene mutated',
                                                text_font_size='8pt',
                                                text_align='left'))

# plot the toggle switch protein concentrations for tog11 and tog21
onemut_togprot_figure.line(onemut_ts - tf_mut[0], onemut_xs[:, circuit_name2pos['p_tog11']], line_width=2,
                           line_color=bkRGB(100, 149, 237),
                           legend_label='tog11')
onemut_togprot_figure.line(onemut_ts - tf_mut[0], onemut_xs[:, circuit_name2pos['p_tog21']], line_width=2,
                           line_color=bkRGB(207, 181, 59),
                           legend_label='tog21')

# plot the toggle switch protein concentrations for tog12 and tog22
onemut_togprot_figure.line(onemut_ts - tf_mut[0], onemut_xs[:, circuit_name2pos['p_tog12']], line_width=2,
                           line_color=bkRGB(127, 255, 212),
                           legend_label='tog12')
onemut_togprot_figure.line(onemut_ts - tf_mut[0], onemut_xs[:, circuit_name2pos['p_tog22']], line_width=2,
                           line_color=bkRGB(252, 194, 0),
                           legend_label='tog22')

# legend formatting
onemut_togprot_figure.legend.location = "bottom_right"
onemut_togprot_figure.legend.label_text_font_size = "8pt"
onemut_togprot_figure.legend.padding = 2
onemut_togprot_figure.legend.spacing = 1
onemut_togprot_figure.legend.glyph_width = 10
onemut_togprot_figure.legend.margin = 7

# set fonts
onemut_togprot_figure.xaxis.axis_label_text_font_size = "8pt"
onemut_togprot_figure.xaxis.major_label_text_font_size = "8pt"
onemut_togprot_figure.yaxis.axis_label_text_font_size = "8pt"
onemut_togprot_figure.yaxis.major_label_text_font_size = "8pt"
onemut_togprot_figure.yaxis[0].formatter = bkmodels.PrintfTickFormatter(format="%.1e")

# FIGS3D - GROWTH RATE AND CAT PROTEIN CONCENTRATION
# get the reference growth rate
pos_premut = np.where(onemut_ts <= tf_mut[0])[0][-1]
l_ref = onemut_ls[pos_premut]

onemut_grcat_figure = bkplot.figure(
    frame_width=200,
    frame_height=125,
    title="Toggle switches with co-expression",
    x_axis_label="Time since mutation, h",
    y_axis_label="Relative cell growth rate",
    x_range=(-5, tf_mut[1] - tf_mut[0]),
    y_range=(0.9, 1.05),
    tools="box_zoom,pan,hover,reset,save"
)

onemut_grcat_figure.add_layout(
    bkmodels.PolyAnnotation(xs=[0, 0, tf_mut[1] - tf_mut[0], tf_mut[1] - tf_mut[0]],
                            ys=[0, 1.75, 1.75, 0],
                            line_width=0, line_alpha=0,
                            fill_color=bkRGB(100, 100, 100, 0.25)))
onemut_grcat_figure.add_layout(bkmodels.Label(x=0, y=1.5,
                                           x_offset=2, y_offset=-12,
                                           text='tog11 gene mutated',
                                           text_font_size='8pt',
                                           text_align='left'))
# plot the growth rate
onemut_grcat_figure.line(onemut_ts - tf_mut[0], onemut_ls/l_ref, line_width=2, line_color=bkRGB(0, 0, 0),
                      legend_label='Growth rate')

# create an extra  y range for plotting CAT protein concentrations
onemut_grcat_figure.extra_y_ranges = {"p_cat": bkmodels.Range1d(start=0, end=2500)}
onemut_grcat_figure.add_layout(bkmodels.LinearAxis(y_range_name="p_cat",
                                     axis_label="CAT conc., nM",
                                     axis_line_color=bkRGB(222, 49, 99),
                                     major_tick_line_color=bkRGB(222, 49, 99),
                                     minor_tick_line_color=bkRGB(222, 49, 99)),
                 'right')  # add the alternative axis label to the figure

# plot the CAT protein concentrations
onemut_grcat_figure.line(onemut_ts - tf_mut[0], onemut_xs[:, circuit_name2pos['p_cat']], line_width=2, line_color=bkRGB(222, 49, 99),
           y_range_name="p_cat", legend_label="CAT conc.")

# set fonts
onemut_grcat_figure.xaxis.axis_label_text_font_size = "8pt"
onemut_grcat_figure.xaxis.major_label_text_font_size = "8pt"
onemut_grcat_figure.yaxis.axis_label_text_font_size = "8pt"
onemut_grcat_figure.yaxis.major_label_text_font_size = "8pt"

# legend formatting
onemut_grcat_figure.legend.location = "right"
onemut_grcat_figure.legend.label_text_font_size = "8pt"
onemut_grcat_figure.legend.padding = 3
onemut_grcat_figure.legend.spacing = 1
onemut_grcat_figure.legend.glyph_width = 10
onemut_grcat_figure.legend.margin = 7


# show plots
onemut_togprot_figure.output_backend = "svg"
onemut_grcat_figure.output_backend = "svg"
bkplot.show(bklayouts.column([onemut_togprot_figure, onemut_grcat_figure]))